In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

import numpy as np
import torch
from ipywidgets import interact
import matplotlib.pyplot as plt
import mne

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [2]:
from research.data import *
import torchvision.transforms as T
import torch.nn.functional as F

transform = T.Compose([
    T.Resize(256),
    T.ToTensor(),
])

things_dataset = ThingsDataset(
    root="X:\\Datasets\\EEG\\Things-concepts-and-images\\",
    transform=transform,
    supplementary_path="X:\\Datasets\\EEG\\Things-supplementary\\",
    latent_name="bigbigan-resnet50",
)

things_eeg_dataset = ThingsEEG(
    things_dataset, 
    things_eeg_path="X:\\Datasets\\EEG\\Things-EEG1\\",
    #source="eeglab",
    window=(-0.1, 0.2),
    #stimulus_window = (-10, 1),
    include_participants=['sub-34'],
    folds=(0,1,2,3,4,"test"),
    verbose=True,
)


Extracting parameters from X:\Datasets\EEG\Things-EEG1\sub-34\data\sub-34\eeg\sub-34_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3576359  =      0.000 ...  3576.359 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upp

In [ ]:
from random import Random
from pipeline.compact_json_encoder import CompactJSONEncoder

folds = {}
participant_ids = list(things_eeg_dataset.participant_data['participant_id'])
for i, participant_id in enumerate(participant_ids):
    block_ids = list(range(12))
    Random(i).shuffle(block_ids)
    grouped_ids = zip(block_ids[:6], block_ids[6:])
    folds[participant_id] = dict(zip(['0', '1', '2', '3', '4', 'test'], grouped_ids))

out_path = "X:\\Datasets\\EEG\\Things-supplementary\\eeg-split.json"
json_encoder = CompactJSONEncoder(indent=2)
with Path(out_path).open(mode="w") as f:
    out_str = json_encoder.encode(folds)
    f.write(out_str)

In [3]:
%matplotlib qt

@interact(participant_id=list(things_eeg_dataset.participants.keys()))
def show(participant_id):
    participant = things_eeg_dataset.participants[participant_id]
    raw = participant['raw']

    @interact(event_id=(0, len(participant['rsvp_events']) - 1))
    def get_event(event_id):
        epochs = participant['epochs']
        event = participant['rsvp_events'].loc[event_id]
        print(event)
        #image = things_dataset[str(event['image_name'])]
        #print(image['unique_image_name'])
        epoch = epochs[event_id]
        epoch.plot(n_channels=128)
        
    #raw.plot_psd(fmax=50)
    #raw.plot(start=5, duration=1, n_channels=30)

interactive(children=(Dropdown(description='participant_id', options=('sub-34',), value='sub-34'), Output()), …

In [4]:
participant = list(things_eeg_dataset.participants.values())[0]
print(participant['participant_id'])
raw = participant['raw']
epochs = participant['epochs']
data = np.stack([epoch[:, :].astype(np.float32) for epoch in epochs])

latents = np.stack([
    things_dataset.images_map[image_name]['latents']['z_mean']
    for image_name in participant['rsvp_events']['image_name']
])
latents.shape


sub-34


(24648, 120)

In [ ]:
X.shape

In [5]:
N, C, T = data.shape
X = data[:, :,]
X = data.reshape(N, C * T)
y = latents

In [6]:
from sklearn.model_selection import train_test_split
train_indices, test_indices = train_test_split(np.arange(N), train_size=0.8)

In [7]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import AdaBoostRegressor

print(X.shape)
model = LinearRegression()
#model = Ridge(alpha=1e-8)
#model = MultiOutputRegressor(LinearSVR())
#model = MLPRegressor(hidden_layer_sizes=(240,), verbose=True, learning_rate_init=1,)
#model = MultiOutputRegressor(AdaBoostRegressor())
#model = MultiOutputRegressor(SVR())

model.fit(X[train_indices], y[train_indices])
print(model.score(X[train_indices], y[train_indices]), model.score(X[test_indices], y[test_indices]))

(24648, 18900)
0.6132345792143935 -2.2315217244778225
Dropped 0 epochs: 
Channels marked as bad: none


In [8]:
import io
import os
import sys
import IPython.display
import PIL.Image
from pprint import pformat

import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from research.scripts.bigbigan import *

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
module_path = 'https://tfhub.dev/deepmind/bigbigan-resnet50/1'  # ResNet-50
module = hub.Module(module_path)

for signature in module.get_signature_names():
    print('Signature:', signature)
    print('Inputs:', pformat(module.get_input_info_dict(signature)))
    print('Outputs:', pformat(module.get_output_info_dict(signature)))
    print()
    
bigbigan = BigBiGAN(module)

enc_ph = bigbigan.make_encoder_ph()
gen_ph = bigbigan.make_generator_ph()
gen_samples = bigbigan.generate(gen_ph, upsample=True)
recon_x = bigbigan.reconstruct_x(enc_ph, upsample=True)
enc_features = bigbigan.encode(enc_ph, return_all_features=True)
disc_scores_enc = bigbigan.discriminate(*bigbigan.enc_pairs_for_disc(enc_ph))
disc_scores_gen = bigbigan.discriminate(*bigbigan.gen_pairs_for_disc(gen_ph))
losses = bigbigan.losses(enc_ph, gen_ph)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Signature: generate
Inputs: {'z': <hub.ParsedTensorInfo shape=(?, 120) dtype=float32 is_sparse=False>}
Outputs: {'default': <hub.ParsedTensorInfo shape=(?, 128, 128, 3) dtype=float32 is_sparse=False>,
 'upsampled': <hub.ParsedTensorInfo shape=(?, 256, 256, 3) dtype=float32 is_sparse=False>}

Signature: default
Inputs: {'x': <hub.ParsedTensorInfo shape=(?, 256, 256, 3) dtype=float32 is_sparse=False>}
Outputs: {'default': <hub.ParsedTensorInfo shape=(?, 120) dtype=float32 is_sparse=False>}

Signature: encode
Inputs: {'x': <hub.ParsedTensorInfo shape=(?, 256, 256, 3) dtype=float32 is_sparse=False>}
Outputs: {'avepool_feat': <hub.ParsedTensorInfo shape=(?, 2048) dtype=float32 is_sparse=False>,
 'bn_crelu_feat': <hub.ParsedTensorInfo shape=(?, 4096) dtype=float32 is_sparse=False>,
 'default': <hub.ParsedTensorInfo shape=(?, 120) dtype=float32 is_sparse=False>,
 'z_mean': <hub.ParsedTensorInfo shape=(?, 120) dtype=float32 is_sparse=False>,
 'z_sample': <hub.ParsedTensorInfo shape=(?, 120) dt

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
@interact(batch_id=(0, 20))
def show(batch_id):
    sample_ids = train_indices[batch_id * 16:(batch_id + 1) * 16]

    feed_dict = {gen_ph: y[sample_ids]}
    encoded_images = sess.run(gen_samples, feed_dict=feed_dict)

    feed_dict = {gen_ph: model.predict(X[sample_ids])}
    reconstructed_images = sess.run(gen_samples, feed_dict=feed_dict)

    image_names = participant['rsvp_events'].loc[sample_ids]['image_name']
    print(list(image_names))
    stimulus_images = np.stack([
        things_dataset[image_name]['data']
        for image_name in image_names
    ])
    stimulus_images = np.moveaxis(stimulus_images, 1, -1) * 2 - 1
    images = interleave(stimulus_images, encoded_images, reconstructed_images)
    imshow(imgrid(image_to_uint8(images), cols=6))
    #imshow(imgrid(image_to_uint8(target_images), cols=4))

interactive(children=(IntSlider(value=10, description='batch_id', max=20), Output()), _dom_classes=('widget-in…

In [ ]:
encoded_images.shape

In [ ]:
imshow(imgrid(image_to_uint8(np.moveaxis(images, 1, -1) * 2 - 1), cols=4))